In [40]:
import numpy as np
import torch as pt
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import collections as coll
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import cm
import gc
import numpy as np
import torch as pt
import torch
import torch.nn as nn
import torch

def print_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated() / 1024 ** 3
    cached_memory = torch.cuda.memory_reserved() / 1024 ** 3
    print(f"Allocated GPU Memory: {allocated_memory:.2f} GB")
    print(f"Cached GPU Memory: {cached_memory:.2f} GB")

print_gpu_memory()
import time as TIME
import torch_tensorrt
from FHNCUDAlib import FHNCUDA
import numpy as np

import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

loaded_module = nn.Linear(10,10)
st=torch.load('network.0.pth')
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit


pt.set_grad_enabled (False) 
numinputs=1
numoutputs=2
class Net(nn.Module):
    def __init__(self, numinputs, numoutputs, numlayers=4, H=10):
        super(Net, self).__init__()

        self.layers = nn.ModuleList()
        self.layers.append(nn.utils.weight_norm(nn.Linear(numinputs, H), name='weight', dim=0).cuda())

        for _ in range(numlayers - 1):
            self.layers.append(nn.utils.weight_norm(nn.Linear(H, H), name='weight', dim=0).cuda())

        self.final_layer = nn.Linear(H, numoutputs).cuda()

        for layer in self.layers:
            layer.eval()
        self.final_layer.eval()

    def forward(self, x):
        for layer in self.layers:
            x = f.silu(layer(x))

        return self.final_layer(x)

    def load(self, od):
        for k, v in od.items():
            if k.startswith('_impl.layers'):
                layer_num = int(k.split('.')[2])
                layer = self.layers[layer_num]
                if k.endswith('linear.weight'):
                    layer.weight_v.data = v
                    layer.weight_v.requires_grad = False
                elif k.endswith('linear.weight_g'):
                    layer.weight_g.data = v
                    layer.weight_g.requires_grad = False
                elif k.endswith('linear.bias'):
                    layer.bias.data = v
                    layer.bias.requires_grad = False
            elif k == '_impl.final_layer.linear.weight':
                self.final_layer.weight.data = v
                self.final_layer.weight.requires_grad = False
            elif k == '_impl.final_layer.linear.bias':
                self.final_layer.bias.data = v
                self.final_layer.bias.requires_grad = False

    def __prepare_scriptable__(self):
        for layer in self.layers:
            for hook in layer._forward_pre_hooks.values():
                if hook.__module__ == "torch.nn.utils.weight_norm" and hook.__class__.__name__ == "WeightNorm":
                    torch.nn.utils.remove_weight_norm(layer)
        return self

import itertools



def Modelrun_s(x,M):
   
    gc.collect()
    torch.cuda.empty_cache()
    my2dspace = pt.tensor(x, requires_grad=False).float().cuda()
    M.eval()
    start_time = TIME.time()
    myOutput = M(my2dspace)
    torch.cuda.synchronize()  # Wait for the events to be recorded!
    reftime = TIME.time()- start_time
    myCPUOutput = myOutput.cpu()


    uu = myCPUOutput.numpy()

    #print('uu: ', uu.T[0])

    myCPUOutput.squeeze().detach().numpy()
    gc.collect()
    torch.cuda.empty_cache()
    return uu,reftime


def Modelrun(x, M, batch_size=10*20*10*200):

    my2dspace = x
    M.eval()
        
    num_samples = my2dspace.shape[0]
  
    gc.collect()
    torch.cuda.empty_cache()
    uu_list = []
   

    reftime =0
    
    gc.collect()
    torch.cuda.empty_cache()
    print("batchs ",num_samples//batch_size)

    for i in range(0, num_samples, batch_size):

        
        
        
        #print_gpu_memory()

        batch_input =torch.tensor(my2dspace[i:i+batch_size], requires_grad=False).float().cuda() 
        
        
       # print_gpu_memory()

        
        
        start_time = TIME.time()
        

        batch_output = M(batch_input)
        
        
 

        torch.cuda.synchronize()  # Wait for the events to be recorded!
        
        
        
        
        reftime =reftime+ TIME.time() - start_time
        
        
        uu_list.append(batch_output.cpu().numpy())
        #del batch_input
        #del batch_output
        #print_gpu_memory()
       
 
    uu = np.concatenate(uu_list)
    
    return uu, reftime


def compileRT(modelTorch):
    gc.collect()
    torch.cuda.empty_cache()
    X=np.zeros((2,5))
    model = torch.jit.script(modelTorch).eval()  # torch module needs to be in eval (not training) mode



    trt_ts_module = torch_tensorrt.compile(modelTorch, inputs = [torch_tensorrt.Input(
            min_shape=(1*100,4),
            opt_shape=(10*1024*100,4),
        
            max_shape=(40000*100,4),
            dtype=torch.float32)],
        enabled_precisions = torch.float16, # Run with FP32
        #workspace size referece a quantidade de memoria usado na build phase, removi para alocar toda a memoria do device, deve acelerar o processo
    )


    gc.collect()
    torch.cuda.empty_cache()
    return trt_ts_module


print("Compiling")
net =Net(4,2,6,32)


PATH=""
od=pt.load('network.0.pth')
net.load(od)
net=net.cuda()


#model =  compileRT(net)
print("Done compiling")

print("Memory",print_gpu_memory())

# The flag below controls whether to allow TF32 on matmul. This flag defaults to False
# in PyTorch 1.12 and later.
print(torch.backends.cuda.matmul.allow_tf32)
#F32 on cuDNN. This flag defaults to True.
print(torch.backends.cudnn.allow_tf32)
print(net)




batch_size=batch_size=1024*100
x=torch.rand((batch_size,4)).cuda()
# Export the model
torch.onnx.export(net,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "model.onnx",   # where to save the model (can be a file or file-like object)
                 # export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'],
           )






Allocated GPU Memory: 0.01 GB
Cached GPU Memory: 0.07 GB
Compiling
Done compiling
Allocated GPU Memory: 0.01 GB
Cached GPU Memory: 0.07 GB
Memory None
True
True
Net(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1-5): 5 x Linear(in_features=32, out_features=32, bias=True)
  )
  (final_layer): Linear(in_features=32, out_features=2, bias=True)
)
=========== Diagnostic Run torch.onnx.export version 2.1.0a0+4136153 ===========
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [42]:
!/opt/tensorrt/bin/trtexec  --onnx=model.onnx --saveEngine=model.trt  --fp16 --useSpinWait 


&&&& RUNNING TensorRT.trtexec [TensorRT v8601] # /opt/tensorrt/bin/trtexec --onnx=model.onnx --saveEngine=model.trt --fp16 --useSpinWait
[11/05/2023-02:00:37] [I] === Model Options ===
[11/05/2023-02:00:37] [I] Format: ONNX
[11/05/2023-02:00:37] [I] Model: model.onnx
[11/05/2023-02:00:37] [I] Output:
[11/05/2023-02:00:37] [I] === Build Options ===
[11/05/2023-02:00:37] [I] Max batch: explicit batch
[11/05/2023-02:00:37] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[11/05/2023-02:00:37] [I] minTiming: 1
[11/05/2023-02:00:37] [I] avgTiming: 8
[11/05/2023-02:00:37] [I] Precision: FP32+FP16
[11/05/2023-02:00:37] [I] LayerPrecisions: 
[11/05/2023-02:00:37] [I] Layer Device Types: 
[11/05/2023-02:00:37] [I] Calibration: 
[11/05/2023-02:00:37] [I] Refit: Disabled
[11/05/2023-02:00:37] [I] Version Compatible: Disabled
[11/05/2023-02:00:37] [I] TensorRT runtime: full
[11/05/2023-02:00:37] [I] Lean DLL Path: 
[11/05/2023-02:00:37] [I] Temp